# ============================================================
# NOTEBOOK: 04_lime_analysis.ipynb
# ANÁLISIS DE INTERPRETABILIDAD CON LIME
# ============================================================

In [ ]:
# ============================================================
# Celda 1 - CONFIGURACIÓN INICIAL Y IMPORTS
# ============================================================
"""
# 🔍 Análisis de Interpretabilidad con LIME
## Local Interpretable Model-agnostic Explanations para DistilBERT

Este notebook implementa LIME para interpretar las decisiones del modelo DistilBERT
en la tarea de clasificación de sentimientos y compara con los resultados de SHAP.
"""

import warnings
warnings.filterwarnings('ignore')

# Imports básicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import time
from typing import List, Dict, Any, Tuple
from IPython.display import display, HTML

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Imports de ML
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# LIME
from lime.lime_text import LimeTextExplainer

print("🔧 CONFIGURACIÓN DEL ENTORNO PARA LIME")
print("="*60)
print(f"📦 Versiones de librerías:")
print(f"  • PyTorch: {torch.__version__}")
print(f"  • LIME: Instalado")
print(f"  • Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"  • GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# ============================================================
# Celda 2 - CARGAR MODELO Y DATOS
# ============================================================
print("\n📚 CARGANDO MODELO Y DATOS PARA LIME")
print("="*60)

# Configuración
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"
CACHE_DIR = "./models_cache"
DATA_DIR = "./explainability_analysis"

# Cargar modelo y tokenizer
print("\n1️⃣ Cargando modelo DistilBERT...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)
model.eval()

# Mover a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"✅ Modelo cargado en {device}")

# Crear pipeline para predicciones más fáciles
classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Cargar casos prioritarios (mismos que usamos con SHAP)
print("\n2️⃣ Cargando casos prioritarios...")
try:
    import glob
    import os
    
    priority_files = glob.glob(f"{DATA_DIR}/priority_cases_*.json")
    if priority_files:
        latest_file = max(priority_files, key=os.path.getctime)
        with open(latest_file, 'r', encoding='utf-8') as f:
            priority_cases = json.load(f)
        print(f"✅ Cargados {len(priority_cases)} casos prioritarios")
    else:
        # Casos de ejemplo
        priority_cases = [
            {
                'text': "This movie was absolutely fantastic! Best film I've seen all year.",
                'true_label': 1,
                'predicted_label': 1,
                'confidence': 0.99,
                'category': 'high_confidence_correct'
            },
            {
                'text': "Terrible movie. Complete waste of time.",
                'true_label': 0,
                'predicted_label': 0,
                'confidence': 0.98,
                'category': 'high_confidence_correct'
            }
        ]
        print(f"⚠️ Usando {len(priority_cases)} casos de ejemplo")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# ============================================================
# Celda 3 - FUNCIÓN DE PREDICCIÓN PARA LIME
# ============================================================
print("\n🔧 PREPARANDO FUNCIÓN DE PREDICCIÓN PARA LIME")
print("="*60)

def predict_for_lime(texts: List[str]) -> np.ndarray:
    """
    Función de predicción compatible con LIME.
    Debe devolver probabilidades para cada clase.
    
    Args:
        texts: Lista de textos a predecir
        
    Returns:
        Array de probabilidades [n_samples, n_classes]
    """
    if isinstance(texts, str):
        texts = [texts]
    
    # Hacer predicciones
    predictions = classifier(list(texts))
    
    # Convertir a formato de probabilidades [neg, pos]
    proba_array = []
    for pred in predictions:
        # El pipeline devuelve LABEL_0 (NEGATIVE) o LABEL_1 (POSITIVE)
        if pred['label'] == 'NEGATIVE' or pred['label'] == 'LABEL_0':
            prob_neg = pred['score']
            prob_pos = 1 - pred['score']
        else:
            prob_pos = pred['score']
            prob_neg = 1 - pred['score']
        proba_array.append([prob_neg, prob_pos])
    
    return np.array(proba_array)

# Verificar función
test_texts = ["This is great!", "This is terrible!"]
test_proba = predict_for_lime(test_texts)
print(f"✅ Función de predicción verificada")
for i, text in enumerate(test_texts):
    print(f"  • '{text}': Neg={test_proba[i][0]:.3f}, Pos={test_proba[i][1]:.3f}")


In [ ]:
# ============================================================
# Celda 4 - INICIALIZAR LIME EXPLAINER
# ============================================================
print("\n🎯 INICIALIZANDO LIME EXPLAINER")
print("="*60)

# Configurar LIME Text Explainer
lime_explainer = LimeTextExplainer(
    class_names=['NEGATIVE', 'POSITIVE'],
    split_expression=r'\s+',  # Separar por espacios
    bow=False,  # No usar bag of words, mantener orden
    random_state=42
)

print("✅ LIME Explainer configurado")
print(f"\n📊 Configuración:")
print(f"  • Clases: {lime_explainer.class_names}")
print(f"  • Método: Perturbación de texto")
print(f"  • Split: Por espacios")
print(f"  • Random state: 42 (reproducible)")

In [ ]:
# ============================================================
# Celda 5 - ANÁLISIS DE UN CASO SIMPLE
# ============================================================
print("\n🔍 ANÁLISIS DE CASO SIMPLE CON LIME")
print("="*60)

# Texto de ejemplo (mismo que usamos con SHAP)
simple_text = "This movie is absolutely terrible and boring."
print(f"\n📝 Texto: '{simple_text}'")

# Predicción original
pred = classifier(simple_text)[0]
print(f"🎯 Predicción: {pred['label']} (confianza: {pred['score']:.3f})")

# Calcular explicación LIME
print("\n⏳ Calculando explicación LIME...")
print("  (Generando perturbaciones del texto...)")

start_time = time.time()
lime_explanation = lime_explainer.explain_instance(
    simple_text,
    predict_for_lime,
    num_features=10,  # Top 10 palabras
    num_samples=1000   # Número de perturbaciones
)
calc_time = time.time() - start_time

print(f"✅ Cálculo completado en {calc_time:.2f} segundos")

# Mostrar palabras más importantes
print(f"\n📊 Top palabras influyentes para clase POSITIVE:")
for word, importance in lime_explanation.as_list(label=1)[:5]:
    emoji = "🟢" if importance > 0 else "🔴"
    print(f"  {emoji} '{word}': {importance:.4f}")

# Verificar si hay explicación para clase NEGATIVE
print(f"\n📊 Top palabras influyentes para predicción:")
# LIME por defecto explica la clase predicha
prediction_proba = predict_for_lime([simple_text])[0]
predicted_class = np.argmax(prediction_proba)

if predicted_class == 1:
    print("  (Explicación para clase POSITIVE)")
    for word, importance in lime_explanation.as_list()[:5]:
        emoji = "🟢" if importance > 0 else "🔴"
        print(f"  {emoji} '{word}': {importance:.4f}")
else:
    print("  (Explicación para clase NEGATIVE)")
    for word, importance in lime_explanation.as_list()[:5]:
        emoji = "🟢" if importance > 0 else "🔴"
        print(f"  {emoji} '{word}': {importance:.4f}")



In [ ]:
# ============================================================
# Celda 6 - VISUALIZACIÓN DE LIME
# ============================================================
print("\n📊 VISUALIZACIONES DE LIME")
print("="*60)

# Obtener datos para visualización
exp_data = lime_explanation.as_list()  # Todas las palabras
words = [x[0] for x in exp_data[:10]]
values = [x[1] for x in exp_data[:10]]

# Crear figura con 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Visualización 1: Bar plot horizontal
colors = ['green' if v > 0 else 'red' for v in values]
ax1.barh(range(len(words)), values, color=colors, alpha=0.7)
ax1.set_yticks(range(len(words)))
ax1.set_yticklabels(words)
ax1.set_xlabel('Importancia LIME')
ax1.set_title(f'Top 10 palabras más influyentes')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.grid(True, alpha=0.3)

# Visualización 2: Bar plot vertical con valores absolutos
abs_values = [abs(v) for v in values]
sorted_indices = np.argsort(abs_values)[::-1][:10]
sorted_words = [words[i] for i in sorted_indices]
sorted_values = [abs_values[i] for i in sorted_indices]
sorted_colors = [colors[i] for i in sorted_indices]

ax2.bar(range(len(sorted_words)), sorted_values, color=sorted_colors, alpha=0.7)
ax2.set_xticks(range(len(sorted_words)))
ax2.set_xticklabels(sorted_words, rotation=45, ha='right')
ax2.set_ylabel('|Importancia|')
ax2.set_title('Importancia absoluta (magnitud)')
ax2.grid(True, alpha=0.3, axis='y')

plt.suptitle(f'LIME: Análisis de "{simple_text}"', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Información adicional
print(f"\n📊 Resumen de la explicación:")
print(f"  • Palabras analizadas: {len(exp_data)}")
print(f"  • Palabras con impacto positivo: {sum(1 for _, v in exp_data if v > 0)}")
print(f"  • Palabras con impacto negativo: {sum(1 for _, v in exp_data if v < 0)}")
print(f"  • Palabra más influyente: '{words[0]}' ({values[0]:.4f})")

In [ ]:
# ============================================================
# Celda 7 - ANÁLISIS DE MÚLTIPLES CASOS
# ============================================================
print("\n🔍 ANÁLISIS DE MÚLTIPLES CASOS CON LIME")
print("="*60)

# Analizar los mismos casos que con SHAP
n_cases = min(5, len(priority_cases))
cases_to_analyze = priority_cases[:n_cases]

print(f"\n📋 Analizando {n_cases} casos prioritarios...")
print("-"*40)

# Almacenar resultados
lime_results = []

for i, case in enumerate(cases_to_analyze, 1):
    text = case['text']
    true_label = case['true_label']
    
    print(f"\nCaso {i}/{n_cases}:")
    print(f"  Texto: '{text[:80]}...'")
    print(f"  Categoría: {case['category']}")
    print(f"  Label real: {true_label}")
    
    # Calcular LIME
    print("  ⏳ Calculando LIME...")
    start = time.time()
    
    # Variar num_samples según la longitud del texto
    text_length = len(text.split())
    num_samples = min(2000, max(500, text_length * 50))
    
    explanation = lime_explainer.explain_instance(
        text,
        predict_for_lime,
        num_features=min(10, text_length),
        num_samples=num_samples
    )
    elapsed = time.time() - start
    
    # Guardar resultados
    result = {
        'text': text,
        'true_label': true_label,
        'predicted_label': case.get('predicted_label', -1),
        'category': case['category'],
        'explanation': explanation,
        'computation_time': elapsed,
        'num_samples': num_samples
    }
    lime_results.append(result)
    
    print(f"  ✅ Completado en {elapsed:.2f}s (con {num_samples} muestras)")

print(f"\n✅ Análisis completado para {len(lime_results)} casos")
avg_time = np.mean([r['computation_time'] for r in lime_results])
print(f"⏱️ Tiempo promedio por caso: {avg_time:.2f} segundos")

In [ ]:
# ============================================================
# Celda 8 - COMPARACIÓN DE EXPLICACIONES
# ============================================================
print("\n📊 COMPARACIÓN DE EXPLICACIONES LIME ENTRE CASOS")
print("="*60)

# Crear figura comparativa
fig, axes = plt.subplots(len(lime_results), 1, figsize=(12, 4*len(lime_results)))
if len(lime_results) == 1:
    axes = [axes]

for idx, (result, ax) in enumerate(zip(lime_results, axes)):
    # Extraer explicación para clase positiva
    exp_data = result['explanation'].as_list(label=1)[:10]
    
    # Preparar datos
    words = [x[0] for x in exp_data]
    values = [x[1] for x in exp_data]
    
    # Crear bar plot
    colors = ['green' if v > 0 else 'red' for v in values]
    ax.barh(range(len(words)), values, color=colors, alpha=0.7)
    ax.set_yticks(range(len(words)))
    ax.set_yticklabels(words)
    ax.set_xlabel('Importancia LIME (impacto en predicción POSITIVE)')
    ax.set_title(f"Caso {idx+1}: {result['category']} (tiempo: {result['computation_time']:.1f}s)", 
                 fontweight='bold')
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    ax.grid(True, alpha=0.3)

plt.suptitle("LIME: Comparación de Palabras Más Influyentes por Caso", 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Celda 9 - ANÁLISIS DE ESTABILIDAD
# ============================================================
print("\n🔬 ANÁLISIS DE ESTABILIDAD DE LIME")
print("="*60)

# Texto para análisis de estabilidad (mismo que SHAP)
# Usar un caso real ambiguo de tu dataset celda 2 "index": 301, confianza ~51.6%
stability_text = "the x potion gives the quickly named blossom , bubbles and buttercup supernatural powers that include extraordinary strength and laser-beam eyes , which unfortunately do n't enable them to discern flimsy screenplays ."

print(f"\n📝 Texto base: '{stability_text}'")

# Realizar múltiples explicaciones
n_runs = 3
stability_results = []
stability_words = {}

print(f"\n🔄 Ejecutando {n_runs} explicaciones del mismo texto...")
for run in range(n_runs):
    print(f"  Run {run+1}/{n_runs}...", end='')
    
    # LIME con misma configuración pero diferente random state
    temp_explainer = LimeTextExplainer(
        class_names=['NEGATIVE', 'POSITIVE'],
        split_expression=r'\s+',
        random_state=42 + run  # Diferente seed
    )
    
    explanation = temp_explainer.explain_instance(
        stability_text,
        predict_for_lime,
        num_features=10,
        num_samples=1000
    )
    
    # Guardar importancias
    for word, importance in explanation.as_list(label=1):
        if word not in stability_words:
            stability_words[word] = []
        stability_words[word].append(importance)
    
    stability_results.append(explanation)
    print(" ✓")

# Calcular varianza
print(f"\n📊 Análisis de varianza entre runs:")
word_variances = {}
for word, values in stability_words.items():
    if len(values) == n_runs:  # Solo palabras que aparecen en todos los runs
        word_variances[word] = {
            'mean': np.mean(values),
            'std': np.std(values),
            'values': values
        }

# Ordenar por varianza
sorted_words = sorted(word_variances.items(), key=lambda x: x[1]['std'])

print(f"\n🎯 Palabras más estables (menor desviación estándar):")
for word, stats in sorted_words[:3]:
    print(f"  • '{word}': μ={stats['mean']:.4f}, σ={stats['std']:.4f}")

print(f"\n⚠️ Palabras menos estables (mayor desviación estándar):")
for word, stats in sorted_words[-3:]:
    print(f"  • '{word}': μ={stats['mean']:.4f}, σ={stats['std']:.4f}")


In [ ]:
# ============================================================
# Celda 10 - COMPARACIÓN LIME vs SHAP
# ============================================================
print("\n📊 COMPARACIÓN DE TIEMPOS: LIME vs SHAP")
print("="*60)

# Cargar resultados de SHAP si existen
try:
    shap_files = glob.glob(f"{DATA_DIR}/shap_results_*.json")
    if shap_files:
        with open(max(shap_files, key=os.path.getctime), 'r') as f:
            shap_data = json.load(f)
        shap_avg_time = shap_data['metadata']['avg_computation_time']
        print(f"✅ Datos de SHAP cargados")
    else:
        shap_avg_time = 15.0  # Estimado típico
        print(f"⚠️ Usando tiempo estimado para SHAP")
except:
    shap_avg_time = 15.0

# Crear comparación
comparison_data = {
    'Método': ['LIME', 'SHAP'],
    'Tiempo Promedio (s)': [avg_time, shap_avg_time],
    'Speedup': [1.0, shap_avg_time / avg_time]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n📋 Tabla comparativa de tiempos:")
print(df_comparison.to_string(index=False))
print(f"\n⚡ LIME es {shap_avg_time / avg_time:.1f}x más rápido que SHAP")

# Visualizar comparación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico de barras de tiempo
ax1.bar(df_comparison['Método'], df_comparison['Tiempo Promedio (s)'], 
        color=['orange', 'blue'], alpha=0.7)
ax1.set_ylabel('Tiempo (segundos)')
ax1.set_title('Tiempo de Cómputo por Método')
ax1.grid(True, alpha=0.3)

# Gráfico de características
features = ['Velocidad', 'Estabilidad', 'Base Teórica', 'Interpretabilidad']
lime_scores = [9, 6, 7, 8]  # Scores cualitativos (1-10)
shap_scores = [3, 9, 10, 7]

x = np.arange(len(features))
width = 0.35

ax2.bar(x - width/2, lime_scores, width, label='LIME', color='orange', alpha=0.7)
ax2.bar(x + width/2, shap_scores, width, label='SHAP', color='blue', alpha=0.7)
ax2.set_ylabel('Score (1-10)')
ax2.set_xlabel('Característica')
ax2.set_title('Comparación Cualitativa')
ax2.set_xticks(x)
ax2.set_xticklabels(features, rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Celda 11 - EXPORTAR RESULTADOS
# ============================================================
print("\n💾 EXPORTANDO RESULTADOS DE LIME")
print("="*60)

# Preparar datos para exportación
export_data = {
    'metadata': {
        'model': MODEL_NAME,
        'method': 'LIME',
        'n_cases_analyzed': len(lime_results),
        'avg_computation_time': avg_time,
        'avg_samples_used': np.mean([r['num_samples'] for r in lime_results]),
        'timestamp': pd.Timestamp.now().isoformat()
    },
    'cases': []
}

for result in lime_results:
    case_data = {
        'text': result['text'],
        'true_label': result['true_label'],
        'category': result['category'],
        'computation_time': result['computation_time'],
        'num_samples': result['num_samples'],
        'top_positive_words': [],
        'top_negative_words': []
    }
    
    # Extraer palabras más importantes
    exp_list = result['explanation'].as_list(label=1)  # Clase POSITIVE
    
    # Top palabras positivas
    pos_words = [(word, score) for word, score in exp_list if score > 0]
    pos_words.sort(key=lambda x: x[1], reverse=True)
    case_data['top_positive_words'] = [
        {'word': word, 'importance': float(score)}
        for word, score in pos_words[:5]
    ]
    
    # Top palabras negativas
    neg_words = [(word, score) for word, score in exp_list if score < 0]
    neg_words.sort(key=lambda x: x[1])
    case_data['top_negative_words'] = [
        {'word': word, 'importance': float(score)}
        for word, score in neg_words[:5]
    ]
    
    export_data['cases'].append(case_data)

# Guardar JSON
output_path = f"{DATA_DIR}/lime_results_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, ensure_ascii=False, indent=2)

print(f"✅ Resultados guardados en: {output_path}")


In [ ]:
# ============================================================
# Celda 12 - ANÁLISIS DE DIFERENCIAS ENTRE MÉTODOS
# ============================================================
print("\n🔄 ANÁLISIS DE DIFERENCIAS LIME vs SHAP")
print("="*60)

# Ejemplo específico de comparación
test_case = "This movie is terrible but has some good moments."
print(f"\n📝 Texto de prueba: '{test_case}'")

# LIME
print("\n🔶 Explicación con LIME:")
lime_exp = lime_explainer.explain_instance(
    test_case,
    predict_for_lime,
    num_features=5,
    num_samples=500
)

print("Top 5 palabras (LIME):")
for word, imp in lime_exp.as_list(label=1)[:5]:
    print(f"  • '{word}': {imp:.4f}")

# Para comparar con SHAP (si tienes los valores guardados)
print("\n🔷 Para comparar con SHAP:")
print("  → Ejecutar el mismo texto en 03_shap_analysis.ipynb")
print("  → Comparar qué palabras identifica cada método")
print("  → Analizar si hay consenso en las palabras clave")

In [ ]:
# ============================================================
# Celda 13 - RESUMEN Y CONCLUSIONES
# ============================================================
print("\n📊 RESUMEN DEL ANÁLISIS CON LIME")
print("="*60)

print("\n✅ ANÁLISIS COMPLETADO:")
print(f"  • Casos analizados: {len(lime_results)}")
print(f"  • Tiempo promedio: {avg_time:.2f} segundos/caso")
print(f"  • Muestras promedio: {np.mean([r['num_samples'] for r in lime_results]):.0f}")
print(f"  • Tiempo total: {sum(r['computation_time'] for r in lime_results):.2f} segundos")

print("\n🔍 OBSERVACIONES CLAVE:")
print("  1. LIME es significativamente más rápido que SHAP (~10x)")
print("  2. Las explicaciones pueden variar entre ejecuciones (estocástico)")
print("  3. Funciona bien para textos cortos y medianos")
print("  4. La interpretación es intuitiva (importancia de palabras)")

print("\n💡 VENTAJAS DE LIME:")
print("  ✓ Muy rápido (1-3 segundos por texto)")
print("  ✓ Agnóstico al modelo (funciona con cualquier clasificador)")
print("  ✓ Visualizaciones intuitivas")
print("  ✓ Fácil de entender para usuarios no técnicos")

print("\n⚠️ LIMITACIONES DE LIME:")
print("  ✗ Resultados estocásticos (no determinísticos)")
print("  ✗ Aproximación local puede no ser precisa")
print("  ✗ Sensible al número de muestras")
print("  ✗ No tiene garantías teóricas como SHAP")

print("\n📊 COMPARACIÓN FINAL LIME vs SHAP:")

comparison_table = pd.DataFrame({
    'Característica': [
        'Velocidad',
        'Estabilidad',
        'Base Teórica',
        'Precisión',
        'Interpretabilidad',
        'Escalabilidad'
    ],
    'LIME': [
        '⭐⭐⭐⭐⭐',
        '⭐⭐',
        '⭐⭐⭐',
        '⭐⭐⭐',
        '⭐⭐⭐⭐',
        '⭐⭐⭐⭐⭐'
    ],
    'SHAP': [
        '⭐⭐',
        '⭐⭐⭐⭐⭐',
        '⭐⭐⭐⭐⭐',
        '⭐⭐⭐⭐⭐',
        '⭐⭐⭐',
        '⭐⭐'
    ]
})

print("\n", comparison_table.to_string(index=False))

print("\n🎯 RECOMENDACIONES DE USO:")
print("\n  📌 Usa LIME cuando:")
print("     • Necesitas explicaciones rápidas")
print("     • Trabajas con muchos textos")
print("     • La audiencia no es técnica")
print("     • Necesitas un prototipo rápido")

print("\n  📌 Usa SHAP cuando:")
print("     • Necesitas garantías matemáticas")
print("     • La precisión es crítica")
print("     • Requieres consistencia absoluta")
print("     • Haces análisis académico/investigación")

print("\n🚀 PRÓXIMOS PASOS:")
print("  → Crear visualización comparativa lado a lado")
print("  → Analizar casos donde LIME y SHAP difieren más")
print("  → Evaluar híbrido: LIME para exploración, SHAP para validación")
print("  → Implementar dashboard interactivo con ambos métodos")

print("\n" + "="*60)
print("✅ Notebook de análisis LIME completado exitosamente")